**Demonstrate the effect of Interference Mitigation on the Cross-Ambiguity Function**

In [1]:
# Load the required libraries
import numpy as np

import ipyvolume as ipv
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

# Custom library for the generation of the C/A code and for signal acquisition
import cafwidget as caf

**Set the signal characteristics**

In [2]:
signal_char = {
               "fs" : 10e6,          # Sampling frequency
               "fi" : 0e3,         # Intermediate frequency
              }

isComplex = True   # Tell if the samples are in I/Q format
is16Bits  = False  # Tell if the samples are stored on 16 bits/byte
isRtl = False      # Tell if the data were collected using an RTl-SDR dongle

**Acquisition parameters**

In [3]:
acq_char = {
            "Nd" : 121,              # Number of Doppler bins
            "DopStep" : 125,        # Doppler bin size in Hz
            "K" : 1                 # Number of non-coherent integrations
           }

**Open the input file and read the data**

In [4]:
# Create and display a FileChooser widget
fc = FileChooser()
display(fc)

FileChooser(path='/home/daniele/Documents/Projects/2021/Jamming demo', filename='', title='HTML(value='', layo…

In [5]:
filename = fc.selected
if filename is None :
    filename = "./JammerData.bin"
    
#open the binary file
fid = open(filename,'rb');

secondOfData = 0.01
N = np.int(np.floor( secondOfData * signal_char["fs"] ))

if isComplex :
    if is16Bits :    
        data = np.fromfile(fid, dtype = np.int16, count = 2 * N )
    else :
        if isRtl :
            data = np.fromfile(fid, dtype = np.uint8, count = 2 * N )
            data = data.astype(dtype = np.int8) - 127
        else :
            data = np.fromfile(fid, dtype = np.int8, count = 2 * N )
    data = data[::2] + 1j*data[1::2]
else:
    if is16Bits :    
        data = np.fromfile(fid, dtype = np.int16, count = N )
    else :
        data = np.fromfile(fid, dtype = np.int8, count = N )

**Create the CAF object**

In [7]:
# SVID of the satellite signal to acquire
sat = 7

CAF = caf.CAFInteractive(sat, data, signal_char, acq_char )

interactive(children=(FloatSlider(value=5.0, continuous_update=False, description='$T_h$', max=20.0, min=1.0),…

In [ ]:
fdata = np.fft.fft( CAF.data )
import matplotlib.pyplot as plt
plt.plot(abs(fdata))